In [ ]:
import pysam
from collections import Counter
import pandas as pd
samfile = pysam.AlignmentFile("/Users/SemiQuant/Downloads/BDQ_haplo/TB-B240-054_S26-bowtie2.bam", "rb")

# var_dic will be snps/indels, if also adding reference then edit the below
# import numpy as np
# vant = np.loadtxt("/Users/SemiQuant/Bioinformatics/Projects/Hasan/snps.tsv", delimiter="\t", 
#           dtype={'names': ('CHROM', 'POS', 'REF', 'ALT', 'TYPE'),
#           'formats': ('<U100', 'int', 'U10', 'U10', 'U10')}, skiprows=1)
var_dic = {}
# this is gross, but i dont really speak python 
with open("/Users/SemiQuant/Bioinformatics/Projects/Hasan/snps.tsv") as f:
    next(f) # skip header line
    for line in f:
        (CHROM, POS, REF, ALT, TYPE) = line.split()
        try:
            var_dic[CHROM][int(POS)].setdefault('call', []).append(ALT)
        except:
            try:
                var_dic[CHROM][int(POS)] = {'call' : [ALT]}
            except:
                var_dic[CHROM] = {int(POS) : {'call' : [ALT]}}

# var_dic={'rpoC': {1221: {'call': ['T']}}, 'Rv0678_amplicon4': {111: {'call': ['A', 'G', 'G', 'T']}, 112: {'call': ['G', '-']}, 114: {'call': ['C']}, 116: {'call': ['C']}}}var_dic={'rpoC': {1221: {'call': ['T']}}, 'Rv0678_amplicon4': {111: {'call': ['A', 'G', 'G', 'T']}, 112: {'call': ['G', '-']}, 114: {'call': ['C']}, 116: {'call': ['C']}}}

# this is like this as I dont really know pysam and how to do it cleaner, with also getting readnames
for chrom in var_dic:
    if chrom not in samfile.references:
        print(chrom, " not found in bam file!")
    else:
        for i in var_dic[chrom]:
            for pileupcolumn in samfile.pileup(chrom, i, i+1):
                for pileupread in pileupcolumn.pileups:
                    if pileupread.query_position == i:
                        if not pileupread.is_del: # if reference then add
                            read_call_tmp = pileupread.alignment.query_sequence[pileupread.query_position]
                            try:
                                #  check if it matches the call in there as pairs have same name
                                if read_call_tmp == var_dic[chrom][i][pileupread.alignment.query_name]:
                                    break
                                else:
                                    var_dic[chrom][i][pileupread.alignment.query_name] = "N"
                            except:
                                if pileupread.is_refskip == 0: # if reference then add, not sure if this is working
                                    var_dic[chrom][i][pileupread.alignment.query_name] = read_call_tmp
                                elif read_call_tmp in var_dic[chrom][i]["call"]:
                                    var_dic[chrom][i][pileupread.alignment.query_name] = read_call_tmp
                                else:
                                    var_dic[chrom][i][pileupread.alignment.query_name] = "N"
                        else:
                            try:
                                #  check if it matches the call in there as pairs have same name
                                if read_call_tmp == var_dic[chrom][i][pileupread.alignment.query_name]:
                                    break
                                else:
                                    var_dic[chrom][i][pileupread.alignment.query_name] = "N"
                            except:
                                if read_call_tmp == var_dic[chrom][i][pileupread.alignment.query_name]:
                                    break
                                else:
                                    var_dic[chrom][i][pileupread.alignment.query_name] = "-"

In [ ]:
var_df = pd.DataFrame.from_dict({(i,j): var_dic[i][j] 
                           for i in var_dic.keys() 
                           for j in var_dic[i].keys()},
                       orient='columns')
var_df.columns = var_df.columns.to_flat_index()
# var_df.columns = [' '.join(str(col)).strip() for col in var_df.columns.values]
var_df = var_df[1:]
cnms = []
for col in var_df.columns.values:
    cnms.append(col[0] + "_" + str(col[1]))
var_df.columns = cnms

In [ ]:
import pysam
import numpy as np
from collections import Counter
vant = np.loadtxt("/Users/SemiQuant/Bioinformatics/Projects/Hasan/snps.tsv", delimiter="\t", 
          dtype={'names': ('CHROM', 'POS', 'REF', 'ALT', 'TYPE'),
          'formats': ('<U100', 'int', 'U10', 'U10', 'U10')}, skiprows=1)

# input file is the reference name, reference based positions and mutations and type

samfile = pysam.AlignmentFile("/Users/SemiQuant/Downloads/test/test.bam", "rb")

# TODO
# add in procedures for indels
# add in better output format, make it a matrix
# add in reference sequence so can determine more info later


phase_out = []
for chrom in set(vant["CHROM"]):
    seq_out = []
    pos = vant["POS"][vant["CHROM"] == chrom]
    call = vant["ALT"][vant["CHROM"] == chrom]
    ref = vant["REF"][vant["CHROM"] == chrom]
    try:
        for read in samfile.fetch(chrom, until_eof=True):
            try:
                #convert to positions in the read
                pos_conv = read.query_alignment_start + read.get_reference_positions()[0] #unaligned leading bases + position of first alignment on reference
                pos_conv = pos - pos_conv
                phase = ""
                call_c = 0
                for p in pos_conv:
                    try:
                        # we dont want any bases that were filtered out so only take those that match the input
                        if read.query_sequence[p] in call[call_c].split(',') or read.query_sequence[p] == ref[call_c]: # in case more than on allele
                            phase += read.query_sequence[p]
                        else:
                            phase += "N"
                    except Exception:
                        phase += "N"
                    call_c += 1
                seq_out.append(phase)
            except Exception:
                continue
        # phase_out.append([[chrom], [Counter(seq_out)]])
        phase_out.append([[chrom], list(Counter(seq_out).items())])
    except Exception:
        print(chrom, " not found in bam file!")

In [ ]:
phase_out

In [ ]:
import pysam
import numpy as np
from collections import Counter
samfile = pysam.AlignmentFile("/Users/SemiQuant/Downloads/BDQ_haplo/TB-B240-054_S26-bowtie2.bam", "rb")


In [ ]:
count=0
for read in samfile.fetch(until_eof=True):
    count+=1
    if read.qname == 'M02061:126:000000000-BVK9B:1:1102:15579:11359':
        for cigar_operations in read.cigar:
            (type_cigar_op,length_cigar_op)=cigar_operations
            print(cigar_operations)
            print(type_cigar_op)

#             print(read.query_sequence[p])
#                 if type_cigar_op==2 :# DELETION
#         for p in range(0, 240):
#             print(read.query_alignment_start)
#             print(p)
#             print(read.query_sequence[p])
                    
#     if count >10:
#         break

In [ ]:
for pileupcolumn in samfile.pileup('Rv0678_amplicon4', 111, 112):
    for pileupread in pileupcolumn.pileups:
        #so this is each base in the read in the pileup
        if pileupread.alignment.query_name == 'M02061:126:000000000-BVK9B:1:1102:15579:11359':
#             print(pileupread)
#             print(pileupread.query_position)
#             print ('\tbase in read %s = %s' %
#                   (pileupread.alignment.query_name,
#                    pileupread.alignment.query_sequence[pileupread.query_position]))

In [ ]:
import pysam
from collections import Counter
import pandas as pd
samfile = pysam.AlignmentFile("/Users/SemiQuant/Downloads/BDQ_haplo/TB-B240-054_S26-bowtie2.bam", "rb")

# var_dic will be snps/indels, if also adding reference then edit the below
# import numpy as np
# vant = np.loadtxt("/Users/SemiQuant/Bioinformatics/Projects/Hasan/snps.tsv", delimiter="\t", 
#           dtype={'names': ('CHROM', 'POS', 'REF', 'ALT', 'TYPE'),
#           'formats': ('<U100', 'int', 'U10', 'U10', 'U10')}, skiprows=1)
var_dic = {}
with open("/Users/SemiQuant/Bioinformatics/Projects/Hasan/snps.tsv") as f:
    next(f) # skip header line
    for line in f:
        (CHROM, POS, REF, ALT, TYPE) = line.split()
        try:
            var_dic[CHROM][int(POS)].setdefault('call', []).append(ALT)
        except:
            try:
                var_dic[CHROM][int(POS)] = {'call' : [ALT]}
            except:
                var_dic[CHROM] = {int(POS) : {'call' : [ALT]}}


In [ ]:
var_dic

In [ ]:
for chrom in var_dic:
    if chrom not in samfile.references:
        print(chrom, " not found in bam file!")
    else:
        for i in var_dic[chrom]:
            for pileupcolumn in samfile.pileup(chrom, i, i+1):
                for pileupread in pileupcolumn.pileups:
                    if pileupread.query_position == i:
                        if not pileupread.is_del: # if reference then add
                            read_call_tmp = pileupread.alignment.query_sequence[pileupread.query_position]
                            try:
                                #  check if it matches the call in there as pairs have same name
                                if read_call_tmp == var_dic[chrom][i][pileupread.alignment.query_name]:
                                    break
                                else:
                                    var_dic[chrom][i][pileupread.alignment.query_name] = "N"
                            except:
                                if pileupread.is_refskip == 0: # if reference then add, not sure if this is working
                                    var_dic[chrom][i][pileupread.alignment.query_name] = read_call_tmp
                                elif read_call_tmp in var_dic[chrom][i]["call"]:
                                    var_dic[chrom][i][pileupread.alignment.query_name] = read_call_tmp
                                else:
                                    var_dic[chrom][i][pileupread.alignment.query_name] = "N"
                        else:
                            try:
                                #  check if it matches the call in there as pairs have same name
                                if read_call_tmp == var_dic[chrom][i][pileupread.alignment.query_name]:
                                    break
                                else:
                                    var_dic[chrom][i][pileupread.alignment.query_name] = "N"
                            except:
                                if read_call_tmp == var_dic[chrom][i][pileupread.alignment.query_name]:
                                    break
                                else:
                                    var_dic[chrom][i][pileupread.alignment.query_name] = "-"


                                    

In [ ]:
var_df = pd.DataFrame.from_dict({(i,j): var_dic[i][j] 
                           for i in var_dic.keys() 
                           for j in var_dic[i].keys()},
                       orient='columns')

In [ ]:
var_df

In [ ]:
var_df[1:]

In [ ]:
import pysam
import numpy as np
from collections import Counter
vant = np.loadtxt("/Users/SemiQuant/Bioinformatics/Projects/Hasan/snps.tsv", delimiter="\t", 
          dtype={'names': ('CHROM', 'POS', 'REF', 'ALT', 'TYPE'),
          'formats': ('<U100', 'int', 'U10', 'U10', 'U10')}, skiprows=1)


In [ ]:
vant

In [ ]:
var_dic

In [ ]:
chrom="rpoB"

In [ ]:
var_dic[chrom][i]

In [1]:
import pysam
from collections import Counter
import pandas as pd
samfile = pysam.AlignmentFile("/Users/SemiQuant/Bioinformatics/Projects/Hasan/test/test2/test_pairedBWA_sorted.bam", "rb")

# var_dic will be snps/indels, if also adding reference then edit the below
# import numpy as np
# vant = np.loadtxt("/Users/SemiQuant/Bioinformatics/Projects/Hasan/snps.tsv", delimiter="\t", 
#           dtype={'names': ('CHROM', 'POS', 'REF', 'ALT', 'TYPE'),
#           'formats': ('<U100', 'int', 'U10', 'U10', 'U10')}, skiprows=1)
var_dic = {}
# this is gross, but i dont really speak python 
with open("/Users/SemiQuant/Bioinformatics/Projects/Hasan/test/test2/snps.tsv") as f:
    next(f) # skip header line
    for line in f:
        (CHROM, POS, REF, ALT, TYPE) = line.split()
        try:
            var_dic[CHROM][int(POS)].setdefault('call', []).append(ALT)
        except:
            try:
                var_dic[CHROM][int(POS)] = {'call' : [ALT]}
            except:
                var_dic[CHROM] = {int(POS) : {'call' : [ALT]}}


In [2]:
var_dic

{'rv0678_500': {490: {'call': ['A']}, 874: {'call': ['C']}}}

In [3]:
# this is like this as I dont really know pysam and how to do it cleaner, with also getting readnames
for chrom in var_dic:
    if chrom not in samfile.references:
        print(chrom, " not found in bam file!")
    else:
        for i in var_dic[chrom]:
            for pileupcolumn in samfile.pileup(chrom, i, i+1):
                for pileupread in pileupcolumn.pileups:
                    try:
                        if pileupread.alignment.get_reference_positions()[pileupread.query_position] == i:
                        # if pileupread.query_position == i:
#                             print(not pileupread.is_del)
                            if not pileupread.is_del: # if reference then add
                                read_call_tmp = pileupread.alignment.query_sequence[pileupread.query_position]
                                try:
                                    #  check if it matches the call in there as pairs have same name
                                    if read_call_tmp == var_dic[chrom][i][pileupread.alignment.query_name]:
#                                         print("a")
                                        break
                                    else:
                                        var_dic[chrom][i][pileupread.alignment.query_name] = "N"
                                except:
                                    if pileupread.is_refskip:
                                        var_dic[chrom][i][pileupread.alignment.query_name] = read_call_tmp
#                                         print("1")
                                    elif read_call_tmp in var_dic[chrom][i]["call"]:
                                        print(var_dic[chrom][i][pileupread.alignment.query_name])
                                        print(read_call_tmp)
                                        var_dic[chrom][i][pileupread.alignment.query_name] = read_call_tmp
                                        print("2")
                                    else:
                                        var_dic[chrom][i][pileupread.alignment.query_name] = "N"
#                                         print("3")
                            else:
                                try:
                                    #  check if it matches the call in there as pairs have same name
                                    if read_call_tmp == var_dic[chrom][i][pileupread.alignment.query_name]:
#                                         print("4")
                                        break
                                    else:
                                        var_dic[chrom][i][pileupread.alignment.query_name] = "N"
#                                         print("5")
                                except:
                                    if read_call_tmp == var_dic[chrom][i][pileupread.alignment.query_name]:
#                                         print("6")
                                        break
                                    else:
                                        var_dic[chrom][i][pileupread.alignment.query_name] = "-"
#                                         print("7")
                    except:
                        pass


In [ ]:
print("test")

In [4]:

# var_df = pd.DataFrame.from_dict(var_dic)
# var_df.reset_index(drop=True, inplace=True)
# var_df = var_df[1:]
var_df = pd.DataFrame.from_dict({(i,j): var_dic[i][j] 
                           for i in var_dic.keys() 
                           for j in var_dic[i].keys()},
                       orient='columns')
var_df.columns = var_df.columns.to_flat_index()
# var_df.columns = [' '.join(str(col)).strip() for col in var_df.columns.values]
var_df = var_df[1:]

In [5]:
var_df

,"(rv0678_500, 490)","(rv0678_500, 874)"
M02061:126:000000000-BVK9B:1:2109:27058:6047,N,NaN
M02061:126:000000000-BVK9B:1:2108:29250:16269,N,NaN
M02061:126:000000000-BVK9B:1:1114:3021:7422,N,NaN
M02061:126:000000000-BVK9B:1:2113:9325:3958,N,NaN
M02061:126:000000000-BVK9B:1:2108:17271:16043,N,NaN
...,...,...
M02061:126:000000000-BVK9B:1:1109:5121:13900,NaN,N
M02061:126:000000000-BVK9B:1:1109:5237:7883,NaN,N
M02061:126:000000000-BVK9B:1:1109:5445:18025,NaN,N
M02061:126:000000000-BVK9B:1:1109:5503:14303,NaN,N


In [ ]:
for chrom in var_dic:
    if chrom not in samfile.references:
        print(chrom, " not found in bam file!")
    else:
        for i in var_dic[chrom]:
            for pileupcolumn in samfile.pileup(chrom, i, i+1):
                for pileupread in pileupcolumn.pileups:
                    if not pileupread.is_refskip:
                        print("test")




In [3]:
print("test")

test


In [2]:
import pysam
from collections import Counter
import pandas as pd
samfile = pysam.AlignmentFile("/Users/semiquant/Bioinformatics/Projects/Hasan/test/test1.bam", "rb")

# var_dic will be snps/indels, if also adding reference then edit the below
# import numpy as np
# vant = np.loadtxt("/Users/SemiQuant/Bioinformatics/Projects/Hasan/snps.tsv", delimiter="\t", 
#           dtype={'names': ('CHROM', 'POS', 'REF', 'ALT', 'TYPE'),
#           'formats': ('<U100', 'int', 'U10', 'U10', 'U10')}, skiprows=1)
var_dic = {}
# this is gross, but i dont really speak python 
# with open("/Users/SemiQuant/Bioinformatics/Projects/Hasan/snps.tsv") as f:
with open("/Users/SemiQuant/Bioinformatics/Projects/Hasan/test/test2/snps.tsv") as f:
    next(f) # skip header line
    for line in f:
        (CHROM, POS, REF, ALT, TYPE) = line.split()
        try:
            var_dic[CHROM][int(POS)].setdefault('call', []).append(ALT)
        except:
            try:
                var_dic[CHROM][int(POS)] = {'call' : [ALT]}
            except:
                var_dic[CHROM] = {int(POS) : {'call' : [ALT]}}

# var_dic={'rpoC': {1221: {'call': ['T']}}, 'Rv0678_amplicon4': {111: {'call': ['A', 'G', 'G', 'T']}, 112: {'call': ['G', '-']}, 114: {'call': ['C']}, 116: {'call': ['C']}}}var_dic={'rpoC': {1221: {'call': ['T']}}, 'Rv0678_amplicon4': {111: {'call': ['A', 'G', 'G', 'T']}, 112: {'call': ['G', '-']}, 114: {'call': ['C']}, 116: {'call': ['C']}}}

# this is like this as I dont really know pysam and how to do it cleaner, with also getting readnames
for chrom in var_dic:
    if chrom not in samfile.references:
        print(chrom, " not found in bam file!")
    else:
        for i in var_dic[chrom]:
            for pileupcolumn in samfile.pileup(chrom, i, i+1):
                for pileupread in pileupcolumn.pileups:
                    try:
                        if pileupread.alignment.get_reference_positions()[pileupread.query_position] == i:
                        # if pileupread.query_position == i:
                            if not pileupread.is_del: # if reference then add
                                read_call_tmp = pileupread.alignment.query_sequence[pileupread.query_position]
                                try:
                                    #  check if it matches the call in there as pairs have same name
                                    if read_call_tmp == var_dic[chrom][i][pileupread.alignment.query_name]:
                                        break
                                    else:
                                        var_dic[chrom][i][pileupread.alignment.query_name] = "N"
                                except:
                                    if read_call_tmp in var_dic[chrom][i]["call"]:
                                        var_dic[chrom][i][pileupread.alignment.query_name] = read_call_tmp
                                    else:
                                        var_dic[chrom][i][pileupread.alignment.query_name] = "N"
                                # elif not pileupread.is_refskip: #if reference then add, not sure if this is working, it isnt so adding the reference to the dic?
                                #         var_dic[chrom][i][pileupread.alignment.query_name] = read_call_tmp
                            else:
                                try:
                                    #  check if it matches the call in there as pairs have same name
                                    if read_call_tmp == var_dic[chrom][i][pileupread.alignment.query_name]:
                                        break
                                    else:
                                        var_dic[chrom][i][pileupread.alignment.query_name] = "N"
                                except:
                                    if read_call_tmp == var_dic[chrom][i][pileupread.alignment.query_name]:
                                        break
                                    else:
                                        var_dic[chrom][i][pileupread.alignment.query_name] = "-"
                    except:
                        pass

# var_df = pd.DataFrame.from_dict(var_dic)
# var_df.reset_index(drop=True, inplace=True)
# var_df = var_df[1:]
var_df = pd.DataFrame.from_dict({(i,j): var_dic[i][j] 
                           for i in var_dic.keys() 
                           for j in var_dic[i].keys()},
                       orient='columns')
var_df.columns = var_df.columns.to_flat_index()
# var_df.columns = [' '.join(str(col)).strip() for col in var_df.columns.values]
var_df = var_df[1:]

# make this happen in the above rather
cnms = []
for col in var_df.columns.values:
    cnms.append(col[0] + "_" + str(col[1]))

var_df.columns = cnms

[E::hts_open_format] Failed to open file "/Users/SemiQuant/Downloads/BDQ_haplo/TB-B240-054_S26-bowtie2.bam" : No such file or directory


FileNotFoundError: [Errno 2] could not open alignment file `/Users/SemiQuant/Downloads/BDQ_haplo/TB-B240-054_S26-bowtie2.bam`: No such file or directory

In [ ]:
var_df.to_pickle("/Users/semiquant/Bioinformatics/Projects/Hasan/test/var_df2.pkl")